## SGDRegressor

In [ ]:
sdgr = SGDRegressor(max_iter=200, tol=0.00001)

y_living_meters = data_reg[(data_reg['living_meters'].notnull()) & (data_reg['kitchen_meters'].notnull())]['living_meters']
X_living_meters = data_reg[(data_reg['living_meters'].notnull()) & (data_reg['kitchen_meters'].notnull())].iloc[:,[1,3]]

sdgr.fit(X_living_meters.values,y_living_meters.values)

y_pred = sdgr.predict(X_living_meters.values)
resid = y_living_meters - y_pred
# тест на автокорреляцию остатков
sm.stats.stattools.durbin_watson(resid)

In [ ]:
# гомоскедастичность остатков
for i in range(0,2):
    plt.figure(figsize= (3,3))
    plt.scatter(X_living_meters.iloc[:,i].values, resid.values, alpha=0.5)
    plt.xlabel(f'X_living_meters{i+1}')
    plt.ylabel('Остатки')
    plt.title(f'График остатков для X_living_meters{i+1}')
plt.show()

In [ ]:
mae = mean_absolute_error(y_living_meters.values, y_pred)
mse = mean_squared_error(y_living_meters.values, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_living_meters.values, y_pred)
print('mae: %.3f \nmse: %.3f \nrmse: %.3f \nr2:%.3f'%(mae,mse,rmse,r2))

In [ ]:
# распределение ошибок
plt.figure(figsize= (3,3))
plt.hist(resid,bins = 20);

## Градиентный бустинг

In [ ]:
y = data[(data['kitchen_meters'].notnull())]['kitchen_meters'].copy()
X = data[(data['kitchen_meters'].notnull())].loc[:,['total_meters','price','year_of_construction','living_meters','city']].copy()
label_encoder = LabelEncoder()
for item in ['city']:
    X[item] = label_encoder.fit_transform(X[item])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
i = 1
n = X_train.shape[0]
print(n)
while i < 3:
    n //=3
    print(n)
    i+=1
print(f'нужно использовать {n} с factor = 3')

model = GradientBoostingRegressor()
params = {'loss':['squared_error', 'absolute_error'],
          'max_features':[1.0],
          'max_depth':np.logspace(1.3, 2.2, num=4, endpoint=True, base=5, dtype=int).tolist(),
         'learning_rate': [0.005,0.01,0.02],
         'n_estimators':np.logspace(2, 3, num=4, endpoint=True, base=5, dtype=int).tolist(),
         'min_samples_split':np.logspace(0.65, 3, num=4, endpoint=True, base=5, dtype=int).tolist()}
halving_search = HalvingGridSearchCV(estimator=model,
                           param_grid=params,
                           cv=5,
                           aggressive_elimination=True,
                           min_resources= n,
                           factor= 3) # n_jobs = -1
halving_search.fit(X_train, y_train)

print(halving_search.best_params_)
y_pred = halving_search.predict(X_test)
mae = mean_absolute_error(y_test, y_pred) 
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print('mae: %.3f \nmse: %.3f \nrmse: %.3f \nr2:%.3f'%(mae,mse,rmse,r2))